In [22]:
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from Energy.HelpFunctions.get_energy_data import get_energy_data, prepare_data
from HelpFunctions.date_and_time import most_recent_thursday, split_time
from Energy.Models.baseline import baseline
from HelpFunctions.calc_score import evaluate_horizon
from HelpFunctions.mix_models import mix_models
from Energy.Models.Model1 import model1
from Energy.Models.Model2 import model2
from Energy.Models.Model4_population import model4_population
from Energy.Models.Model4 import model4
from Energy.Models.Model3 import model3
from Energy.Models.Model5 import model5
from Energy.Models.Model4_holidays_2 import model4_holidays_2
from Energy.Models.Model4_sunhours import model4_sunhours
# import importlib
# importlib.reload(Energy.Models)

If needed: Fetch energy-data first

In [23]:
from Energy.HelpFunctions.get_energy_data import fetch_energy_data
fetch_energy_data()

  0%|          | 0/259 [00:00<?, ?it/s]/Users/elias/Desktop/PTSFC/Energy/HelpFunctions/get_energy_data.py:55: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  energydata = pd.concat([energydata, pd.DataFrame(rawdata, columns=col_names)])
100%|██████████| 259/259 [00:54<00:00,  4.76it/s]


In [24]:
df = get_energy_data()
df = prepare_data(df)

# Cross validate baseline model

Remove everything til last thursday night 12pm

In [25]:
start_date_excl = most_recent_thursday(df)
df_cval = df.loc[df.index < start_date_excl]

Repeatedly run the model. Record predictions and true values (observations). Make sure the observations are available for the most recent prediction.

In [26]:
from HelpFunctions.mix_models import mix_models_per_horizon


def evaluate_models(models, df, last_x, years =False, months=False, weeks=False):
    # Check that exactly one of the boolean parameters is True
    if sum([years, months, weeks]) != 1:
        raise ValueError("Exactly one of the boolean parameters (years, months, weeks) must be True.")
    
    years = int(years)
    months = int(months)
    weeks = int(weeks)
        
    for m in models:
        print(f'*********** Start the evaluation of Model {m["name"]} ***********')
        m['evaluation'] = evaluate_model(m, df, last_x, years, months, weeks)
        
def evaluate_model(model, df, last_x, years, months, weeks):
    df_before = df
    evaluation = pd.DataFrame()
    
    for w in range(last_x):
        print(f'Iteration {w} of {last_x}')
        df_before, df_after = split_time(df_before, num_years=years, num_months=months, num_weeks=weeks)        
        
        pred = None     
        # Is mixed model?
        if callable(model['function']):
            pred = model['function'](df_before)
        else:
            pred = mix_models_per_horizon(model['function'][0], model['function'][1], df_before)
               
        
        obs = pd.DataFrame({'gesamt': df.loc[pred['forecast_date']]["gesamt"]})
        pred = pred.set_index('forecast_date')
        merged_df = pd.merge(pred, obs, left_index=True, right_index=True) 
    
    
         # Add scores to the merged_df
        for index, row in merged_df.iterrows():
            quantile_preds = row[['q0.025','q0.25','q0.5','q0.75','q0.975']]
            observation = row['gesamt']
            score = evaluate_horizon(quantile_preds, observation)
            merged_df.at[index, 'score'] = score
        # print(merged_df[['q0.025','q0.25','q0.5','q0.75','q0.975']])
        evaluation = pd.concat([evaluation, merged_df])
    return evaluation

## Evaluation of selected Models

In [27]:
weights_m5_bl_m4_x = [[1,1,1,0,0,0.3],[0,0,0,0.25,0.25,0.7],[0,0,0,0.25,0.25,0],[0,0,0,0.25,0.25,0],[0,0,0,0.25,0.25,0]]
functions_m5_bl_m4_x = [model5, baseline, model4_population, model4_holidays_2, model4_sunhours]

models = [
    # {
    #     'name': 'mixed',
    #     'function': [[model4_sunhours, model4_holidays_2, model4_population],[1,1,1]]
    # },
    # {
    #     'name': 'm5_bl_m4_x',
    #     'function': [functions_m5_bl_m4_x,weights_m5_bl_m4_x]
    # },
    # {
    #     'name': 'model5',
    #     'function': model5
    # },
    {
        'name': 'baseline',
        'function': baseline
     },
    # {
    #     'name': 'model1',
    #     'function': model1
    # },
    # {
    #     'name': 'model2',
    #     'function': model2
    # },
    # {
    #     'name': 'model3',
    #     'function': model3
    # },
    # {
    #     'name': 'model4',
    #     'function': model4
    # },
    # {
    #     'name': 'model4_sunhours',
    #     'function': model4_sunhours
    # },
    # {
    #     'name': 'model4_holidays_2',
    #     'function': model4_holidays_2
    # },
    # {
    #     'name': 'model4_population',
    #     'function': model4_population
    # },
]

In [28]:
evaluate_models(models, df_cval, last_x=50, weeks=True)

*********** Start the evaluation of Model baseline ***********
Iteration 0 of 50
Iteration 1 of 50
Iteration 2 of 50
Iteration 3 of 50
Iteration 4 of 50
Iteration 5 of 50
Iteration 6 of 50
Iteration 7 of 50
Iteration 8 of 50
Iteration 9 of 50
Iteration 10 of 50
Iteration 11 of 50
Iteration 12 of 50
Iteration 13 of 50
Iteration 14 of 50
Iteration 15 of 50
Iteration 16 of 50
Iteration 17 of 50
Iteration 18 of 50
Iteration 19 of 50
Iteration 20 of 50
Iteration 21 of 50
Iteration 22 of 50
Iteration 23 of 50
Iteration 24 of 50
Iteration 25 of 50
Iteration 26 of 50
Iteration 27 of 50
Iteration 28 of 50
Iteration 29 of 50
Iteration 30 of 50
Iteration 31 of 50
Iteration 32 of 50
Iteration 33 of 50
Iteration 34 of 50
Iteration 35 of 50
Iteration 36 of 50
Iteration 37 of 50
Iteration 38 of 50
Iteration 39 of 50
Iteration 40 of 50
Iteration 41 of 50
Iteration 42 of 50
Iteration 43 of 50
Iteration 44 of 50
Iteration 45 of 50
Iteration 46 of 50
Iteration 47 of 50
Iteration 48 of 50
Iteration 49 of 

In [29]:
models[0]['evaluation']

,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975,gesamt,score
forecast_date,,,,,,,,,
2023-11-24 12:00:00,energy,36 hour,53.274763,60.736375,63.552250,66.767437,74.963481,69.70550,16.129342
2023-11-24 16:00:00,energy,40 hour,50.642019,55.801063,58.491375,62.505938,70.579475,66.67200,20.862060
2023-11-24 20:00:00,energy,44 hour,49.348806,52.374437,56.135375,59.835375,67.135544,63.18650,18.373181
2023-11-25 12:00:00,energy,60 hour,47.477200,51.210125,53.837750,57.057937,62.957481,57.24800,7.488295
2023-11-25 16:00:00,energy,64 hour,44.081156,47.319625,50.045750,53.217188,60.186881,56.81850,17.729442
...,...,...,...,...,...,...,...,...,...
2022-12-16 16:00:00,energy,40 hour,54.849106,58.917375,61.555875,66.352000,72.031300,66.96425,11.209297
2022-12-16 20:00:00,energy,44 hour,51.692600,55.576625,58.415625,64.226063,68.135462,61.90600,8.637237
2022-12-17 12:00:00,energy,60 hour,51.019400,54.218625,55.583500,59.475313,63.994419,61.05350,11.903470


### Save evaluations in pkl file

In [30]:
# with open('./Model evaluations/mm_m5_bl_m4_x.pkl', 'wb') as f:
#     pickle.dump(models, f)

# with open('./Model evaluations/m4_pop_m5.pkl', 'rb') as f:
#     models2 = pickle.load(f)

### Create a table that only contains the different scores of the different models

In [31]:
# scores = [m['evaluation']['score'][m['evaluation']['horizon'] == '36 hour'] for m in models]
# horizons = ['36 hour', '40 hour', '44 hour', '60 hour', '64 hour', '68 hour']
# 
# 
# names = [m['name'] for m in models]
# score_df = pd.concat(scores, axis=1,keys=names)

In [32]:
horizons = ['36 hour', '40 hour', '44 hour', '60 hour', '64 hour', '68 hour']
scores = []
names = []
for h in horizons:
    for m in models:
        col_name = f'{m["name"]}: {h}'
        scores.append(m['evaluation']['score'][m['evaluation']['horizon'] == h])
        names.append(col_name)

score_df = pd.concat(scores, axis=1,keys=names)

In [33]:
score_df

,baseline: 36 hour,baseline: 40 hour,baseline: 44 hour,baseline: 60 hour,baseline: 64 hour,baseline: 68 hour
forecast_date,,,,,,
2022-12-16 12:00:00,6.614797,NaN,NaN,NaN,NaN,NaN
2022-12-16 16:00:00,NaN,11.209297,NaN,NaN,NaN,NaN
2022-12-16 20:00:00,NaN,NaN,8.637237,NaN,NaN,NaN
2022-12-17 12:00:00,NaN,NaN,NaN,11.903470,NaN,NaN
2022-12-17 16:00:00,NaN,NaN,NaN,NaN,20.226258,NaN
...,...,...,...,...,...,...
2023-11-24 16:00:00,NaN,20.862060,NaN,NaN,NaN,NaN
2023-11-24 20:00:00,NaN,NaN,18.373181,NaN,NaN,NaN
2023-11-25 12:00:00,NaN,NaN,NaN,7.488295,NaN,NaN


### Plot the densities of the obtained scores

In [34]:
models_display = ['m5_bl_m4_x']
# models_display = ['baseline', 'model4_sunhours', 'model4_holidays_2', 'model4', 'model4_population']
for h in [str(h) + " hour" for h in [36, 40, 44, 60, 64, 68]]:
    # sns.kdeplot(data=score_df.loc[:,[f'baseline: {h}',f'model3: {h}',f'MM_baseline_model3: {h}']], fill=True)
    sns.kdeplot(data=score_df.loc[:,[f'{m}: {h}' for m in models_display]], fill=True)
    
    # Adding labels and title
    plt.xlabel('Score')
    plt.ylabel('Density')
    plt.title(f'Density Plot of Scores: {h} horizon')
    
    # Display the plot
    plt.savefig(f'plots/density_plot_{h.replace(" ", "_")}.png')
    plt.show()

KeyError: "None of [Index(['m5_bl_m4_x: 36 hour'], dtype='object')] are in the [columns]"

In [ ]:
for h in [str(h) + " hour" for h in [36, 40, 44, 60, 64, 68]]:
    # Plot values from two columns over time
    score_df_36_no_na = score_df[score_df[f'{models_display[0]}: {h}'].notna()]
    for c in [f'{m}: {h}' for m in models_display]:
        # plt.plot(score_df_36_no_na.index, score_df_36_no_na['baseline: 36 hour'], label='baseline')
        # plt.plot(score_df_36_no_na.index, score_df_36_no_na['model1: 36 hour'], label='model1')
        # plt.plot(score_df_36_no_na.index, score_df_36_no_na['model2: 36 hour'], label='model2')
        plt.plot(score_df_36_no_na.index, score_df_36_no_na[c], label=c)
        # plt.plot(score_df_36_no_na.index, score_df_36_no_na[c], label='model4')
        # Adding labels and title
    plt.xlabel('time')
    plt.ylabel('score')
    plt.title(f'Comparison of scores over time: {h}')
    plt.ylim(0,50)
    
    # Display legend
    plt.legend()
    plt.savefig(f'plots/line_plot_{h.replace(" ", "_")}.png')

    # Show the plot
    plt.show()

#### Plot Whole evaluation for a model

In [ ]:
[m['name'] for m in models]

In [ ]:


import matplotlib.pyplot as plt

def plot_evaluation(evaluation_data):
    # Assuming your DataFrame is named df
    numeric_columns = evaluation_data.select_dtypes(include='number')
    
    # Plotting
    plt.figure(figsize=(12, 8))
    for column in numeric_columns.columns:
        plt.plot(evaluation_data.index, evaluation_data[column], label=column)
    
    # Adding labels and legend
    plt.xlabel('Forecast Date')
    plt.ylabel('Values')
    plt.title('Line Plot for Numeric Columns')
    plt.legend()
    plt.grid(True)
    plt.savefig(f'plots/evaluation_overview.png')
    plt.show()

plot_evaluation(models[1]['evaluation'][models[1]['evaluation']['horizon'] == '36 hour'])

### Search for outliers (highest scores per horizon)

In [ ]:
models_display = ['model4_holidays_2']
horizons = [36]
for h in [str(h) + " hour" for h in [36, 40, 44, 60, 64, 68]]:
    for c in [f'{m}: {h}' for m in models_display]:
        df_sorted = score_df[c].sort_values().dropna().tail(10)
        
        
        print(f'{c}')
        print(df_sorted)

In [ ]:
[m['name'] for m in models]

In [ ]:
models[5]